# k8s集成metrics-server

## 参考文献

* https://github.com/kubernetes-sigs/metrics-server

## 实践

```shell
kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/download/v0.5.2/components.yaml
```

* 镜像地址替换为阿里云镜像`registry.cn-hangzhou.aliyuncs.com/google_containers/metrics-server:v0.5.2`
* metrics-server启动参数增加`--kubelet-insecure-tls`

## FAQ

Q: metrics-server error because it doesn't contain any IP SANs?

A: 更新`metrics-server` deployment，启动增加`--kubelet-insecure-tls`参数，参考github issue https://github.com/kubernetes-sigs/metrics-server/issues/196

```yaml
apiVersion: apps/v1
kind: Deployment
  labels:
    k8s-app: metrics-server
  name: metrics-server
  namespace: kube-system
spec:
  progressDeadlineSeconds: 600
  replicas: 1
  revisionHistoryLimit: 10
  selector:
    matchLabels:
      k8s-app: metrics-server
  strategy:
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 0
    type: RollingUpdate
  template:
    metadata:
      creationTimestamp: null
      labels:
        k8s-app: metrics-server
    spec:
      containers:
      - args:
        - --cert-dir=/tmp
        - --secure-port=4443
        - --kubelet-preferred-address-types=InternalIP,ExternalIP,Hostname
        - --kubelet-use-node-status-port
        - --metric-resolution=15s
        - --kubelet-insecure-tls
```